# Rendre opérationnel le processus de déploiement du modèle Amazon Personalize de bout en bout à l'aide du kit SDK AWS Step Functions Data Science

1. [Introduction](#Introduction)
2. [Configuration](#Setup)
3. [États Task](#Task-States)
4. [États Wait](#Wait-States)
5. [États Choice](#Choice-States)
6. [Flux](#Workflow)
7. [Générer des recommandations](#Generate-Recommendations)



## Introduction

Ce manuel décrit l'utilisation du kit SDK AWS Step Functions Data Science pour créer et gérer un flux Amazon Personalize. Le kit SDK Step Functions est une bibliothèque open source qui permet aux scientifiques des données de créer et d'exécuter facilement des flux de machine learning à l'aide d'AWS Step Functions. Pour en savoir plus sur le kit SDK Step Functions, reportez-vous à la documentation suivante :
* [AWS Step Functions](https://aws.amazon.com/step-functions/)
* [AWS Step Functions : Guide du développeur](https://docs.aws.amazon.com/step-functions/latest/dg/welcome.html)
* [Kit SDK AWS Step Functions Data Science](https://aws-step-functions-data-science-sdk.readthedocs.io)

Dans ce manuel, nous allons utiliser le kit SDK pour créer des étapes afin de créer des ressources Amazon Personalize, les relier afin de créer un flux et exécuter ce flux dans AWS Step Functions. 

Pour en savoir plus sur Amazon Personalize, reportez-vous à la documentation suivante :

* [Amazon Personalize](https://aws.amazon.com/personalize/)


## Configuration

### Importer les modules requis à partir du kit SDK

In [ ]:
#import sys
#!{sys.executable} -m pip install --upgrade stepfunctions

In [ ]:
import boto3
import json
import numpy as np
import pandas as pd
import time

personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')


import stepfunctions
import logging

from stepfunctions.steps import *
from stepfunctions.workflow import Workflow

stepfunctions.set_stream_logger(level=logging.INFO)

workflow_execution_role = "<Workflow exection role name>" # paste the StepFunctionsWorkflowExecutionRole ARN from above

### Configurer l'emplacement et le nom du fichier S3

In [ ]:
bucket = "<Bucket Name>"       # replace with the name of your S3 bucket
filename = "<File Name>"  # replace with a name that you want to save the dataset under

### Configurer les rôles IAM

#### Créer un rôle d'exécution pour Step Functions

Pour pouvoir créer et exécuter des flux dans Step Functions, vous avez besoin d'un rôle d'exécution.

1. Accédez à la [console IAM](https://console.aws.amazon.com/iam/)
2. Sélectionnez **Roles** (Rôles), puis **Create role** (Créer un rôle).
3. Sous **Choose the service that will use this role** (Choisir le service qui utilisera ce rôle), sélectionnez **Step Functions**
4. Choisissez **Next** (Suivant) jusqu'à ce que vous puissiez entrer un **Role name** (Nom de rôle)
5. Saisissez un nom tel que `StepFunctionsWorkflowExecutionRole`, puis sélectionnez **Create role** (Créer un rôle)


Associez une politique au rôle que vous avez créé. Les étapes suivantes associent une politique qui fournit un accès complet à Step Functions. Toutefois, en guise de bonne pratique, vous ne devez fournir l'accès qu'aux ressources dont vous avez besoin.  

1. Sous l'onglet **Permissions** (Autorisations), cliquez sur **Add inline policy** (Ajouter une politique intégrée)
2. Saisissez ce qui suit dans l'onglet **JSON**

```json
{
    "Version": "2012-10-17",
    "Statement": [
    
        {
            "Effect": "Allow",
            "Action": [
                "personalize:*"
            ],
            "Resource": "*"
        },   

        {
            "Effect": "Allow",
            "Action": [
                "lambda:InvokeFunction"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "iam:PassRole"
            ],
            "Resource": "*",
        },
        {
            "Effect": "Allow",
            "Action": [
                "events:PutTargets",
                "events:PutRule",
                "events:DescribeRule"
            ],
            "Resource": "*"
        }
    ]
}
```

3. Choisissez **Review policy** (Examiner la politique), puis donnez-lui un nom tel que `StepFunctionsWorkflowExecutionPolicy`
4. Choisissez **Create policy** (Créer une politique). Vous serez redirigé vers la page de détails du rôle.
5. Copiez **Role ARN** (ARN du rôle) au-dessus de **Summary** (Résumé)



In [ ]:
lambda_state_role = LambdaStep(
    state_id="create bucket and role",
    parameters={  
        "FunctionName": "stepfunction_create_personalize_role", #replace with the name of the function you created
        "Payload": {  
           "bucket": bucket
        }
    },
    result_path='$'
 
)

lambda_state_role.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_role.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("CreateRoleTaskFailed")
))

#### Attacher une politique au compartiment S3

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
                
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role


#### Créer un rôle Personalize


In [ ]:
iam = boto3.client("iam")

role_name = "<Role Name>" # Create a personalize role


assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)



policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]


## Préparation des données

### Télécharger, préparer et charger les données d'entraînement

In [ ]:
!pwd

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data



In [ ]:
data = data[data['RATING'] > 2]                # keep only movies rated 2 and above
data2 = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] 
data2.to_csv(filename, index=False)

boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

## États Task

### État Task Lambda

Un état `Task` dans Step Functions représente une unité de travail unique effectuée par un flux. Les tâches peuvent appeler des fonctions Lambda et orchestrer d'autres services AWS. Reportez-vous à la section [Intégration des services AWS](https://docs.aws.amazon.com/step-functions/latest/dg/concepts-service-integrations.html) dans le *Guide du développeur AWS Step Functions*.

Les étapes qui suivent permettent de créer un [LambdaStep](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/compute.html#stepfunctions.steps.compute.LambdaStep) appelé `lambda_state`, puis de configurer les options afin de [Réessayer](https://docs.aws.amazon.com/step-functions/latest/dg/concepts-error-handling.html#error-handling-retrying-after-an-error) si la fonction Lambda échoue.

#### Créer une fonction Lambda

Les états Task Lambda de ce flux utilisent la fonction Lambda **(Python 3.x)** qui renvoie des ressources Amazon Personalize telles que Schema (Schéma), Datasetgroup (Groupe de jeux de données), Dataset (Jeu de donées), Solution (Solution), SolutionVersion (Version de la solution), etc. Créez les fonctions suivantes dans la [console Lambda](https://console.aws.amazon.com/lambda/).

1. stepfunction-create-schema
2. stepfunctioncreatedatagroup
3. stepfunctioncreatedataset
4. stepfunction-createdatasetimportjob
5. stepfunction_select-recipe_create-solution
6. stepfunction_create_solution_version
7. stepfunction_getsolution_metric_create_campaign

Faites un copier-coller du code de la fonction Lambda correspondant à partir du dossier ./Lambda/ dans le référentiel


#### Créer un schéma

In [ ]:
lambda_state_schema = LambdaStep(
    state_id="create schema",
    parameters={  
        "FunctionName": "stepfunction-create-schema", #replace with the name of the function you created
        "Payload": {  
           "input": "personalize-stepfunction-schema263"
        }
    },
    result_path='$'    
)

lambda_state_schema.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_schema.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("CreateSchemaTaskFailed")
))

#### Créer un groupe de jeux de données

In [ ]:
lambda_state_datasetgroup = LambdaStep(
    state_id="create dataset Group",
    parameters={  
        "FunctionName": "stepfunctioncreatedatagroup", #replace with the name of the function you created
        "Payload": {  
           "input": "personalize-stepfunction-dataset-group", 
           "schemaArn.$": '$.Payload.schemaArn'
        }
    },

    result_path='$'
)



lambda_state_datasetgroup.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))


lambda_state_datasetgroup.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("CreateDataSetGroupTaskFailed")
))

#### Créer un jeu de données

In [ ]:
lambda_state_createdataset = LambdaStep(
    state_id="create dataset",
    parameters={  
        "FunctionName": "stepfunctioncreatedataset", #replace with the name of the function you created
#        "Payload": {  
#           "schemaArn.$": '$.Payload.schemaArn',
#           "datasetGroupArn.$": '$.Payload.datasetGroupArn',
            
            
#        }
        
        "Payload": {  
           "schemaArn.$": '$.schemaArn',
           "datasetGroupArn.$": '$.datasetGroupArn',        
        } 
        
        
    },
    result_path = '$'
)

lambda_state_createdataset.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_createdataset.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("CreateDataSetTaskFailed")
))

#### Créer une tâche d'importation de jeu de données

In [ ]:
lambda_state_datasetimportjob = LambdaStep(
    state_id="create dataset import job",
    parameters={  
        "FunctionName": "stepfunction-createdatasetimportjob", #replace with the name of the function you created
        "Payload": {  
           "datasetimportjob": "stepfunction-createdatasetimportjob",
           "dataset_arn.$": '$.Payload.dataset_arn',
           "datasetGroupArn.$": '$.Payload.datasetGroupArn',
           "bucket_name": bucket,
           "file_name": filename,
           "role_arn": role_arn
            
        }
    },

    result_path = '$'
)

lambda_state_datasetimportjob.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_datasetimportjob.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("DatasetImportJobTaskFailed")
))

#### Créer une recette et une solution

In [ ]:
lambda_state_select_receipe_create_solution = LambdaStep(
    state_id="select receipe and create solution",
    parameters={  
        "FunctionName": "stepfunction_select-recipe_create-solution", #replace with the name of the function you created
        "Payload": {  
           #"dataset_group_arn.$": '$.Payload.datasetGroupArn' 
            "dataset_group_arn.$": '$.datasetGroupArn'
        }
    },
    result_path = '$'
)

lambda_state_select_receipe_create_solution.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_select_receipe_create_solution.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("DatasetReceiptCreateSolutionTaskFailed")
))

#### Créer une version de solution

In [ ]:
lambda_create_solution_version = LambdaStep(
    state_id="create solution version",
    parameters={  
        "FunctionName": "stepfunction_create_solution_version", 
        "Payload": {  
           "solution_arn.$": '$.Payload.solution_arn'           
        }
    },
    result_path = '$'
)

lambda_create_solution_version.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_create_solution_version.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("CreateSolutionVersionTaskFailed")
))

#### Créer une campagne

In [ ]:
lambda_create_campaign = LambdaStep(
    state_id="create campaign",
    parameters={  
        "FunctionName": "stepfunction_getsolution_metric_create_campaign", 
        "Payload": {  
            #"solution_version_arn.$": '$.Payload.solution_version_arn'  
            "solution_version_arn.$": '$.solution_version_arn'
        }
    },
    result_path = '$'
)

lambda_create_campaign.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_create_campaign.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("CreateCampaignTaskFailed")
))

## États Wait

#### Un état `Wait` dans Step Functions attend un laps de temps spécifique. Référez-vous à la section [Attendre](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/states.html#stepfunctions.steps.states.Wait) de la documentation du kit SDK AWS Step Functions Data Science.

#### Attendre que le schéma soit prêt

In [ ]:
wait_state_schema = Wait(
    state_id="Wait for create schema - 5 secs",
    seconds=5
)

#### Attendre que le groupe de jeux de données soit prêt

In [ ]:
wait_state_datasetgroup = Wait(
    state_id="Wait for datasetgroup - 30 secs",
    seconds=30
)

#### Attendre que le jeu de données soit prêt

In [ ]:
wait_state_dataset = Wait(
    state_id="wait for dataset - 30 secs",
    seconds=30
)

#### Attendre que la tâche d'importation de jeux de données soit ACTIVE (ACTIF)

In [ ]:
wait_state_datasetimportjob = Wait(
    state_id="Wait for datasetimportjob - 30 secs",
    seconds=30
)

#### Attendre que la recette soit prête

In [ ]:
wait_state_receipe = Wait(
    state_id="Wait for receipe - 30 secs",
    seconds=30
)

#### Attendre que la version de la solution soit ACTIVE (ACTIF)

In [ ]:
wait_state_solutionversion = Wait(
    state_id="Wait for solution version - 60 secs",
    seconds=60
)

#### Attendre que la campagne soit ACTIVE (ACTIF)

In [ ]:
wait_state_campaign = Wait(
    state_id="Wait for Campaign - 30 secs",
    seconds=30
)



### Vérifier l'état de la tâche Lambda et agir en conséquence

#### Si un état échoue, transférez-le vers l'état `Fail`. Référez-vous à la section [Échec](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/states.html#stepfunctions.steps.states.Fail) de la documentation du kit SDK AWS Step Functions Data Science.

### vérifier l'état du groupe de jeux de données

In [ ]:
lambda_state_datasetgroupstatus = LambdaStep(
    state_id="check dataset Group status",
    parameters={  
        "FunctionName": "stepfunction_waitforDatasetGroup", #replace with the name of the function you created
        "Payload": {  
           "input.$": '$.Payload.datasetGroupArn',
           "schemaArn.$": '$.Payload.schemaArn'
        }
    },
    result_path = '$'
)

lambda_state_datasetgroupstatus.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_datasetgroupstatus.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("DatasetGroupStatusTaskFailed")
))

### vérifier l'état de la tâche d'importation de jeux de données

In [ ]:
lambda_state_datasetimportjob_status = LambdaStep(
    state_id="check dataset import job status",
    parameters={  
        "FunctionName": "stepfunction_waitfordatasetimportjob", #replace with the name of the function you created
        "Payload": {  
           "dataset_import_job_arn.$": '$.Payload.dataset_import_job_arn',
           "datasetGroupArn.$": '$.Payload.datasetGroupArn'
        }
    },
    result_path = '$'
)

lambda_state_datasetimportjob_status.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_datasetimportjob_status.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("DatasetImportJobStatusTaskFailed")
))

### vérifier l'état de la version de la solution

In [ ]:

solutionversion_succeed_state = Succeed(
    state_id="The Solution Version ready?"
)

In [ ]:
lambda_state_solutionversion_status = LambdaStep(
    state_id="check solution version status",
    parameters={  
        "FunctionName": "stepfunction_waitforSolutionVersion", #replace with the name of the function you created
        "Payload": {  
           "solution_version_arn.$": '$.Payload.solution_version_arn'           
        }
    },
    result_path = '$'
)

lambda_state_solutionversion_status.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_solutionversion_status.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("SolutionVersionStatusTaskFailed")
))

### vérifier l'état de la campagne

In [ ]:
lambda_state_campaign_status = LambdaStep(
    state_id="check campaign status",
    parameters={  
        "FunctionName": "stepfunction_waitforCampaign", #replace with the name of the function you created
        "Payload": {  
           "campaign_arn.$": '$.Payload.campaign_arn'           
        }
    },
    result_path = '$'
)

lambda_state_campaign_status.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_campaign_status.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("CampaignStatusTaskFailed")
))

## États Choice

À présent, associez des branches à l'état Choice que vous avez créé précédemment. Référez-vous à la section *Règles de choix* de la documentation du kit SDK [AWS Step Functions Data Science](https://aws-step-functions-data-science-sdk.readthedocs.io).

#### Relier les étapes à l'aide d'une chaîne pour définir le chemin du flux

La cellule suivante relie les étapes que vous avez créées ci-dessus dans un groupe séquentiel. Le nouveau chemin inclut successivement l'état Lambda, l'état Wait et l'état Succeed que vous avez créés précédemment.

#### Après avoir relié les étapes du chemin du flux à l'aide d'une chaîne, nous allons définir et visualiser le flux

In [ ]:
create_campaign_choice_state = Choice(
    state_id="Is the Campaign ready?"
)

In [ ]:
create_campaign_choice_state.add_choice(
    rule=ChoiceRule.StringEquals(variable=lambda_state_campaign_status.output()['Payload']['status'], value='ACTIVE'),
    next_step=Succeed("CampaignCreatedSuccessfully")     
)
create_campaign_choice_state.add_choice(
    ChoiceRule.StringEquals(variable=lambda_state_campaign_status.output()['Payload']['status'], value='CREATE PENDING'),
    next_step=wait_state_campaign
)
create_campaign_choice_state.add_choice(
    ChoiceRule.StringEquals(variable=lambda_state_campaign_status.output()['Payload']['status'], value='CREATE IN_PROGRESS'),
    next_step=wait_state_campaign
)

create_campaign_choice_state.default_choice(next_step=Fail("CreateCampaignFailed"))


In [ ]:
solutionversion_choice_state = Choice(
    state_id="Is the Solution Version ready?"
)

In [ ]:
solutionversion_succeed_state = Succeed(
    state_id="The Solution Version ready?"
)

In [ ]:
solutionversion_choice_state.add_choice(
    rule=ChoiceRule.StringEquals(variable=lambda_state_solutionversion_status.output()['Payload']['status'], value='ACTIVE'),
    next_step=solutionversion_succeed_state   
)
solutionversion_choice_state.add_choice(
    ChoiceRule.StringEquals(variable=lambda_state_solutionversion_status.output()['Payload']['status'], value='CREATE PENDING'),
    next_step=wait_state_solutionversion
)
solutionversion_choice_state.add_choice(
    ChoiceRule.StringEquals(variable=lambda_state_solutionversion_status.output()['Payload']['status'], value='CREATE IN_PROGRESS'),
    next_step=wait_state_solutionversion
)

solutionversion_choice_state.default_choice(next_step=Fail("create_solution_version_failed"))


In [ ]:
datasetimportjob_succeed_state = Succeed(
    state_id="The Solution Version ready?"
)

In [ ]:
datasetimportjob_choice_state = Choice(
    state_id="Is the DataSet Import Job ready?"
)

In [ ]:
datasetimportjob_choice_state.add_choice(
    rule=ChoiceRule.StringEquals(variable=lambda_state_datasetimportjob_status.output()['Payload']['status'], value='ACTIVE'),
    next_step=datasetimportjob_succeed_state   
)
datasetimportjob_choice_state.add_choice(
    ChoiceRule.StringEquals(variable=lambda_state_datasetimportjob_status.output()['Payload']['status'], value='CREATE PENDING'),
    next_step=wait_state_datasetimportjob
)
datasetimportjob_choice_state.add_choice(
    ChoiceRule.StringEquals(variable=lambda_state_datasetimportjob_status.output()['Payload']['status'], value='CREATE IN_PROGRESS'),
    next_step=wait_state_datasetimportjob
)


datasetimportjob_choice_state.default_choice(next_step=Fail("dataset_import_job_failed"))


In [ ]:
datasetgroupstatus_choice_state = Choice(
    state_id="Is the DataSetGroup ready?"
)

## Flux

### Définir le flux

Dans la cellule suivante, définissez l'étape que vous souhaitez utiliser dans le flux.  Ensuite, créez, visualisez et exécuter ce flux. 

Les étapes se rapportent aux états dans AWS Step Functions. Pour en savoir plus, reportez-vous à la section [États](https://docs.aws.amazon.com/step-functions/latest/dg/concepts-states.html) dans le *Guide du développer AWS Step Functions*. Pour en savoir plus sur les API du kit SDK AWS Step Functions Data Science, rendez-vous sur la page : https://aws-step-functions-data-science-sdk.readthedocs.io. 




### Flux du jeu de donnés

In [ ]:
Dataset_workflow_definition=Chain([lambda_state_schema,
                                   wait_state_schema,
                                   lambda_state_datasetgroup,
                                   wait_state_datasetgroup,
                                   lambda_state_datasetgroupstatus
                                  ])

In [ ]:
Dataset_workflow = Workflow(
    name="Dataset-workflow",
    definition=Dataset_workflow_definition,
    role=workflow_execution_role
)

In [ ]:
Dataset_workflow.render_graph()

In [ ]:
DatasetWorkflowArn = Dataset_workflow.create()

### Flux de l'importation du jeu de données

In [ ]:
DatasetImport_workflow_definition=Chain([lambda_state_createdataset,
                                   wait_state_dataset,
                                   lambda_state_datasetimportjob,
                                   wait_state_datasetimportjob,
                                   lambda_state_datasetimportjob_status,
                                   datasetimportjob_choice_state
                                  ])

In [ ]:
DatasetImport_workflow = Workflow(
    name="DatasetImport-workflow",
    definition=DatasetImport_workflow_definition,
    role=workflow_execution_role
)

In [ ]:
DatasetImport_workflow.render_graph()

In [ ]:
DatasetImportflowArn = DatasetImport_workflow.create()

Flux de la recette et de la solution

In [ ]:
Create_receipe_sol_workflow_definition=Chain([lambda_state_select_receipe_create_solution,
                                   wait_state_receipe,
                                   lambda_create_solution_version,
                                   wait_state_solutionversion,
                                   lambda_state_solutionversion_status,
                                   solutionversion_choice_state
                                  ])

In [ ]:
Create_receipe_sol_workflow = Workflow(
    name="Create_receipe_sol-workflow",
    definition=Create_receipe_sol_workflow_definition,
    role=workflow_execution_role
)

In [ ]:
Create_receipe_sol_workflow.render_graph()

In [ ]:
CreateReceipeArn = Create_receipe_sol_workflow.create()

Créer un flux de campagne

In [ ]:
Create_Campaign_workflow_definition=Chain([lambda_create_campaign,
                                   wait_state_campaign,
                                   lambda_state_campaign_status,
                                   wait_state_datasetimportjob,
                                   create_campaign_choice_state
                                  ])

In [ ]:
Campaign_workflow = Workflow(
    name="Campaign-workflow",
    definition=Create_Campaign_workflow_definition,
    role=workflow_execution_role
)

In [ ]:
Campaign_workflow.render_graph()

In [ ]:
CreateCampaignArn = Campaign_workflow.create()

Flux principal

In [ ]:
call_dataset_workflow_state = Task(
    state_id="DataSetWorkflow",
    resource="arn:aws:states:::states:startExecution.sync:2",
    parameters={
                                "Input": "true",
                                #"StateMachineArn": "arn:aws:states:us-east-1:444602785259:stateMachine:Dataset-workflow",
                                "StateMachineArn": DatasetWorkflowArn
                }
)

In [ ]:
call_datasetImport_workflow_state = Task(
    state_id="DataSetImportWorkflow",
    resource="arn:aws:states:::states:startExecution.sync:2",
    parameters={
                                 "Input":{
                                    "schemaArn.$": "$.Output.Payload.schemaArn",
                                    "datasetGroupArn.$": "$.Output.Payload.datasetGroupArn"
                                   },
                                "StateMachineArn": DatasetImportflowArn,
                }
)

In [ ]:
call_receipe_solution_workflow_state = Task(
    state_id="ReceipeSolutionWorkflow",
    resource="arn:aws:states:::states:startExecution.sync:2",
    parameters={
                                 "Input":{
                                    "datasetGroupArn.$": "$.Output.Payload.datasetGroupArn"

                                   },
                                "StateMachineArn": CreateReceipeArn
                }
)

In [ ]:
call_campaign_solution_workflow_state = Task(
    state_id="CampaignWorkflow",
    resource="arn:aws:states:::states:startExecution.sync:2",
    parameters={
                                 "Input":{
                                    "solution_version_arn.$": "$.Output.Payload.solution_version_arn"

                                   },
                                "StateMachineArn": CreateCampaignArn
                }
)

In [ ]:
Main_workflow_definition=Chain([call_dataset_workflow_state,
                                call_datasetImport_workflow_state,
                                call_receipe_solution_workflow_state,
                                call_campaign_solution_workflow_state
                               ])

In [ ]:
Main_workflow = Workflow(
    name="Main-workflow",
    definition=Main_workflow_definition,
    role=workflow_execution_role
)

In [ ]:
Main_workflow.render_graph()

In [ ]:
Main_workflow.create()

In [ ]:
Main_workflow_execution = Main_workflow.execute()

Main_workflow_execution = Workflow(
    name="Campaign_Workflow",
    definition=path1,
    role=workflow_execution_role
)


In [ ]:
#Main_workflow_execution.render_graph()

### Créer et exécuter le flux

Dans les cellules suivantes, nous allons créer une ramification de flux « happy » dans AWS Step Functions avec [create](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.create), puis l'exécuter avec [execute](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.execute).


In [ ]:
#personalize_workflow.create()

In [ ]:
#personalize_workflow_execution = happy_workflow.execute()

###  Passer en revue la progression du flux

Passez en revue la progression du flux à l'aide de [render_progress](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Execution.render_progress).

Passez en revue l'historique d'exécution en appelant [list_events](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Execution.list_events) pour répertorier tous les événements d'exécution du flux.

In [ ]:
Main_workflow_execution.render_progress()

In [ ]:
Main_workflow_execution.list_events(html=True)

## Générer des recommandations

### À présent que notre campagne a abouti, générons des recommandations pour la campagne

#### Sélectionner un utilisateur et un article

In [ ]:
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1')
items.columns = ['ITEM_ID', 'TITLE']


user_id, item_id, rating, timestamp = data.sample().values[0]

user_id = int(user_id)
item_id = int(item_id)

print("user_id",user_id)
print("items",items)


item_title = items.loc[items['ITEM_ID'] == item_id].values[0][-1]
print("USER: {}".format(user_id))
print("ITEM: {}".format(item_title))
print("ITEM ID: {}".format(item_id))


In [ ]:
wait_recommendations = Wait(
    state_id="Wait for recommendations - 10 secs",
    seconds=10
)

#### Tâche Lambda

In [ ]:
lambda_state_get_recommendations = LambdaStep(
    state_id="get recommendations",
    parameters={  
        "FunctionName": "stepfunction_getRecommendations", 
        "Payload": {  
           "campaign_arn": 'arn:aws:personalize:us-east-1:261602857181:campaign/stepfunction-campaign',            
           "user_id": user_id,  
           "item_id": item_id             
        }
    },
    result_path = '$'
)

lambda_state_get_recommendations.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_get_recommendations.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("GetRecommendationTaskFailed")
    #next_step=recommendation_path   
))

#### Créer un état Succeed

In [ ]:
workflow_complete = Succeed("WorkflowComplete")

In [ ]:
recommendation_path = Chain([ 
lambda_state_get_recommendations,
wait_recommendations,
workflow_complete
])

### Définir, créer, traduire et exécuter le flux de recommandation

Dans les cellules suivantes, nous allons créer un flux dans AWS Step Functions avec [create](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.create), puis l'exécuter avec [execute](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.execute).

In [ ]:
recommendation_workflow = Workflow(
    name="Recommendation_Workflow4",
    definition=recommendation_path,
    role=workflow_execution_role
)



In [ ]:
recommendation_workflow.render_graph()

In [ ]:
recommendation_workflow.create()

In [ ]:
recommendation_workflow_execution = recommendation_workflow.execute()

### Passer en revue la progression

Passez en revue la progression du flux à l'aide de [render_progress](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Execution.render_progress).

Passez en revue l'historique d'exécution en appelant [list_events](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Execution.list_events) pour répertorier tous les événements d'exécution du flux.

In [ ]:
recommendation_workflow_execution.render_progress()

In [ ]:
recommendation_workflow_execution.list_events(html=True)


In [ ]:
item_list = recommendation_workflow_execution.get_output()['Payload']['item_list']

### Obtenir les recommandations

In [ ]:
item_list = recommendation_workflow_execution.get_output()['Payload']['item_list']

print("Recommendations:")
for item in item_list:
    np.int(item['itemId'])
    item_title = items.loc[items['ITEM_ID'] == np.int(item['itemId'])].values[0][-1]
    print(item_title)


## Nettoyer les ressources Amazon Personalize

Assurez-vous de nettoyer les ressources Amazon Personalize et le blog créé par les machines d'état. Connectez-vous à la console Amazon Personalize et supprimez des ressources telles Dataset Groups (Groupes de jeux de données), Dataset (Jeu de données), Solutions (Solutions), Receipts (Recettes) et Campaign (Campagne). 

## Nettoyer les ressources de la machine d'état

In [ ]:
Campaign_workflow.delete()

recommendation_workflow.delete()

Main_workflow.delete()

Create_receipe_sol_workflow.delete()

DatasetImport_workflow.delete()

Dataset_workflow.delete()
